# Plan
- 심볼을 기준으로 크롭하기
- 각 크롭한 이미지를 ocr로 텍스트 추출하기 : paddle
- 텍스트 전체 추출~

### step01 : 크롭하기

# ocr

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
from paddleocr import PaddleOCR

font_path = "/home/dibaeck/.fonts/nanum/NanumGothicCoding.ttf"   # local 폰트 설정
font = ImageFont.truetype(font_path, 15)                        # 폰트 사이즈 조정

# TEST 이미지 
image_path = "../data/easy_task_test05_v1.PNG"

image_paddle = Image.open(image_path).convert("RGB")
draw_paddle = ImageDraw.Draw(image_paddle)

ocr_paddle = PaddleOCR(use_angle_cls=True, lang='korean')
results_paddle = ocr_paddle.predict(image_path)     # 반환정보 좀 달라짐

res = results_paddle[0]
boxes = res.get('rec_boxes', [])    
texts = res.get('rec_texts', [])
scores = res.get('rec_scores', [])

paddle_texts = []

for box, text, score in zip(boxes, texts, scores):
    if text.strip() == "":
        continue  # 빈 텍스트 무시
    # rec_boxes는 [x_min, y_min, x_max, y_max] 형태
    x_min, y_min, x_max, y_max = map(int, box)

    draw_paddle.rectangle([x_min, y_min, x_max, y_max], outline="orange", width=2)
    draw_paddle.text((x_min, y_min - 20), text, fill="brown", font=font)

    paddle_texts.append({
        "text": text,
        "confidence": score,
        "bbox": (x_min, y_min, x_max-x_min, y_max-y_min)
    })

df_paddle = pd.DataFrame(paddle_texts)
# image_paddle.show()

print("\n🟧 PaddleOCR 결과:")
print(df_paddle[['text','confidence']])

/home/dibaeck/anaconda3/envs/proj_v1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_3049442/3309242882.py:14: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  ocr_paddle = PaddleOCR(use_angle_cls=True, lang='korean')
/home/dibaeck/anaconda3/envs/proj_v1/lib/python3.11/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Using official model (PP-LCNet_x1_0_doc_ori), the model files will be automatic

: 